In [5]:
from loguru import logger
import snoop
import birdseye
import numpy as np
%load_ext blackcellmagic

Позднее взыскание: постройте зависимость объёма выданного кредита от возраста, пола, клиентского сегмента, флага проживания в Москве и флага "наличие номера мобильного телефона" с помощью линейной регрессии.

Оцените качество полученной модели (используйте Анализ данных). Отметьте разными цветами те ячейки, на основании которых вы делаете вывод о значимости отдельных коэффициентов.

Воспроизведите руками в Excel расчёты TSS, ESS и RSS, а также R-квадрата и сравните с тем, что вывел Анализ данных.

In [11]:
import scipy

In [ ]:
!pip install line_profiler

In [388]:
import line_profiler

In [1]:
%load_ext line_profiler

In [386]:
snoop.install()

In [3]:
from birdseye import eye
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from IPython.display import display

In [378]:
df

,id_client,id_global,id_city,gender,married,first_time,age,is_educ,cellphone,is_active,cl_segm,amt_loan,date_loan,credit_type,id_city.1,name_city,id_region,name_region,moscow
0,169,3996.0,1.0,0,False,0.0,69.0,1.0,1,1.0,2.0,110000,2021-04-08,POS-1,1.0,0,111.0,Центр,1
1,1622,1771.0,1.0,0,False,1.0,69.0,1.0,0,1.0,2.0,110000,2019-09-14,POS-1,1.0,0,111.0,Центр,1
2,2537,1882.0,1.0,0,False,1.0,39.0,0.0,1,1.0,2.0,70000,2020-03-26,POS-1,1.0,0,111.0,Центр,1
3,799,2500.0,1.0,1,False,0.0,18.0,1.0,1,1.0,2.0,500000,2020-08-08,POS-1,1.0,0,111.0,Центр,1
4,1901,1217.0,1.0,1,False,1.0,53.0,0.0,0,0.0,3.0,900000,2019-10-18,TOP-UP,1.0,0,111.0,Центр,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,1713,2004.0,20.0,1,False,1.0,29.0,0.0,1,1.0,2.0,110000,2020-05-02,POS-1,20.0,0,114.0,Урал,0
2827,476,3189.0,20.0,1,False,1.0,33.0,1.0,1,1.0,2.0,100000,2018-11-23,TOP-UP,20.0,0,114.0,Урал,0
2828,2073,1799.0,20.0,1,False,1.0,71.0,1.0,0,0.0,2.0,70000,2019-05-28,TOP-UP,20.0,0,114.0,Урал,0
2830,2645,1294.0,20.0,1,False,0.0,76.0,1.0,0,1.0,3.0,15000,2019-11-27,POS-1,20.0,0,114.0,Урал,0


In [365]:
df = pd.read_csv("late_collection_clients.csv")
df["cellphone"] = df["cellphone"].apply(lambda x: 1 if x == x else 0)
df["gender"] = df["gender"].apply(lambda x: 1 if x == "F" else 0)

In [112]:
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

In [ ]:
'''@logger.catch
@snoop
@eye'''

In [366]:
df.dropna(inplace=True)
target = df["amt_loan"]

In [372]:
dependent_cols = ["cl_segm", "moscow", "cellphone", "age", "gender"]

In [370]:
df['moscow'] = df['id_city.1'].apply(lambda x: 1 if x == 1 else 0)

In [371]:
df

,id_client,id_global,id_city,gender,married,first_time,age,is_educ,cellphone,is_active,cl_segm,amt_loan,date_loan,credit_type,id_city.1,name_city,id_region,name_region,moscow
0,169,3996.0,1.0,0,False,0.0,69.0,1.0,1,1.0,2.0,110000,2021-04-08,POS-1,1.0,0,111.0,Центр,1
1,1622,1771.0,1.0,0,False,1.0,69.0,1.0,0,1.0,2.0,110000,2019-09-14,POS-1,1.0,0,111.0,Центр,1
2,2537,1882.0,1.0,0,False,1.0,39.0,0.0,1,1.0,2.0,70000,2020-03-26,POS-1,1.0,0,111.0,Центр,1
3,799,2500.0,1.0,1,False,0.0,18.0,1.0,1,1.0,2.0,500000,2020-08-08,POS-1,1.0,0,111.0,Центр,1
4,1901,1217.0,1.0,1,False,1.0,53.0,0.0,0,0.0,3.0,900000,2019-10-18,TOP-UP,1.0,0,111.0,Центр,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,1713,2004.0,20.0,1,False,1.0,29.0,0.0,1,1.0,2.0,110000,2020-05-02,POS-1,20.0,0,114.0,Урал,0
2827,476,3189.0,20.0,1,False,1.0,33.0,1.0,1,1.0,2.0,100000,2018-11-23,TOP-UP,20.0,0,114.0,Урал,0
2828,2073,1799.0,20.0,1,False,1.0,71.0,1.0,0,0.0,2.0,70000,2019-05-28,TOP-UP,20.0,0,114.0,Урал,0
2830,2645,1294.0,20.0,1,False,0.0,76.0,1.0,0,1.0,3.0,15000,2019-11-27,POS-1,20.0,0,114.0,Урал,0


In [2]:
x_train, x_test = df[dependent_cols][:2500], df[dependent_cols][2501:]
y_train, y_test = target[:2500], target[2501:]

NameError: name 'df' is not defined

In [374]:
scaler = StandardScaler()
scaler.fit(x_train, y_train)
scaled_train_data = scaler.transform(x_train)
scaled_test_data = scaler.transform(x_test)

In [380]:
model = LinearRegression().fit(scaled_train_data, y_train)

In [381]:
mean_squared_error(y_test, model.predict(scaled_test_data), squared=False)

414875.3343471405

In [382]:
model.coef_

array([-19752.62593755,  55749.45336162, -17567.28310337, -14531.45200242,
         2201.68840803])

In [383]:
print(model.predict(scaled_test_data)[:10])

[414440.4616875  456250.82305337 348042.65938601 421978.75868876
 418229.49639439 443686.99471794 411528.78794883 465863.20523761
 431123.16638251 421579.85061718]


In [10]:
brak_prob = 0.06 * (1 / 3) + 0.07 * (2 / 3)
0.06 * (1 / 3) / brak_prob

0.29999999999999993

In [26]:
binom = scipy.stats.binom(n=100,p=0.62)

In [28]:
1-binom.cdf(59)

0.6987523768364827

In [21]:
p=0.62
std = np.sqrt(p*(1-p))
k = std/10
2*k

0.09707728879609277